## KAKAOMAP 상세보기 리뷰 스크래핑 코드 ##

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import re
import sys

import warnings
warnings.filterwarnings('ignore')

In [ ]:
## CSV 파일 불러오기
df = pd.read_csv('kakaomap_yeonnam_restaurant_moreview_list.csv') #, encoding='cp949')
df

,사업장명,소재지전체주소,도로명전체주소,좌표정보X(EPSG5174),좌표정보Y(EPSG5174),상세보기url
0,사르르,서울특별시 마포구 연남동 383-100,"서울특별시 마포구 연희로1길 63, 2층 (연남동)",193442.684557,451030.233681,NaN
1,빨강꼬치&꼬떡상회,서울특별시 마포구 연남동 390-71,"서울특별시 마포구 동교로38길 13, 지1층 (연남동)",193303.524530,451089.391094,NaN
2,지라파,서울특별시 마포구 연남동 373-12,"서울특별시 마포구 연남로 8, 1층 (연남동)",193169.497265,450929.132987,NaN
3,돈토키,서울특별시 마포구 연남동 390-28,"서울특별시 마포구 동교로38길 27-19, 지층 B02호 (연남동)",193388.640488,451076.653833,NaN
4,에스유 치즈카페,서울특별시 마포구 연남동 225-29,"서울특별시 마포구 성미산로 175, 지하1층 (연남동)",193333.879086,451308.487923,NaN
...,...,...,...,...,...,...
795,연남제비,서울특별시 마포구 연남동 228-5,"서울특별시 마포구 성미산로 186 (연남동, 1층)",193391.074652,451202.033340,https://place.map.kakao.com/2045155753
796,유키모찌 연남점,서울특별시 마포구 연남동 260-28,"서울특별시 마포구 동교로 240 (연남동,1층)",193255.606492,451004.993797,https://place.map.kakao.com/913195358
797,항저우 샤롱바오,서울특별시 마포구 연남동 568-23,"서울특별시 마포구 동교로27길 41 (연남동,(1층))",192990.763157,450825.627592,https://place.map.kakao.com/961577779
798,뭉텅 연남점,서울특별시 마포구 연남동 228-1,"서울특별시 마포구 성미산로 190 (연남동, 1층)",193406.589452,451183.710005,https://place.map.kakao.com/388449263


In [ ]:
## DF에서 NaN 또는 빈 문자열을 제외한 값만 리스트로 변환
url_list = df['상세보기url'].dropna().tolist()  # NaN 제거
url_list = [url for url in url_list if url.strip()]  # 빈 문자열 제거

# # 결과 출력
# print(url_list)

In [ ]:
## 생성한 리스트에 빈 문자열 또는 NaN 값이 있는지 검증
has_empty_or_nan = any(item == "" or item is None or (isinstance(item, float) and np.isnan(item)) for item in url_list)
print("리스트에 빈 문자열 또는 NaN이 포함되어 있나요?", has_empty_or_nan)

리스트에 빈 문자열 또는 NaN이 포함되어 있나요? False


In [ ]:
## 리스트 갯수(길이) 확인
len(url_list)

691

In [10]:
## 스크래핑 준비

## 리뷰 저장용 DF 생성
review_df = pd.DataFrame(columns=['사업장명', '사업장ID', '리뷰어이름', '리뷰어레벨', '후기개수', '별점평균', '팔로워수', '작성일', '별점', '리뷰내용', '사진URL'])

## 스크래핑용 브라우저 오픈
driver = webdriver.Chrome()
time.sleep(2)

In [11]:
## 카카오맵 검색 가게 후기 스크래핑

## 리뷰 스크래핑 함수
def all_review_scraping(temp_rev, rev_cnt):
    global store_name, store_id
    temp_rev_list = []
    # temp_rev = review_group[0].contents[0]  # 첫 번째 리뷰 콘텐츠

    ## 리뷰 내용 중 더보기 버튼 유무 확인 및 클릭
    if temp_rev.select_one('.btn_more'):
        # print('더보기 버튼이 있습니다!')
        ## 더보기 버튼 클릭
        driver.find_element(By.CSS_SELECTOR, '.desc_review .btn_more').click()
        time.sleep(0.2)

        ## 재 파싱 및 리뷰 목록 다시 가져오기
        temp_html = driver.page_source
        temp_soup = BeautifulSoup(temp_html, 'html.parser')
        # time.sleep(0.2)
        
        review_group = temp_soup.select('#mainContent > div.main_detail > div.detail_cont > div.section_comm.section_review > div.group_review > ul')
        temp_rev = review_group[0].contents[rev_cnt]

    # 1. 리뷰어 이름
    reviewer_name_tag = temp_rev.select_one('.name_user')
    reviewer_name = reviewer_name_tag.contents[1].strip() if reviewer_name_tag and len(reviewer_name_tag.contents) > 1 else "N/A"
    
    # 2. 리뷰어 레벨
    reviewer_level_tag = temp_rev.select_one('.txt_badge')
    reviewer_level = reviewer_level_tag.text.replace('레벨', '').strip() if reviewer_level_tag else "N/A"
    
    # 3. 후기, 별점평균, 팔로워 수 (list_detail 활용)
    review_stats = {}
    list_detail = temp_rev.select_one('.list_detail')
    if list_detail:
        items = list_detail.find_all('li')
        for item in items:
            text = item.text.strip().split(' ')
            if len(text) >= 2:
                key = text[0] # '후기', '별점평균', '팔로워'
                value = text[1] # '162', '4.1', '5'
                review_stats[key] = value

    # 4. 리뷰 작성일
    review_date_tag = temp_rev.select_one('.txt_date')
    review_date = review_date_tag.text.strip() if review_date_tag else "N/A"

    # 5. 별점 (screen_out 클래스 중 두 번째에 텍스트 별점이 있음)
    rating_tag = temp_rev.select('.starred_grade .screen_out')
    # 두 번째 screen_out 태그에 실제 숫자가 들어있음 (별점 1.0)
    rating = rating_tag[1].text.strip() if len(rating_tag) > 1 else "N/A"

    # 6. 리뷰 내용
    review_content_tag = temp_rev.select_one('.desc_review')
    if review_content_tag:
        # <p class="desc_review">텍스트 내용 <span class="btn_more">더보기</span></p>
        # p 태그의 첫 번째 자식 요소가 우리가 원하는 텍스트일 가능성이 높음.
        review_content_parts = [
            str(c) for c in review_content_tag.contents if c.name != 'span' and isinstance(c, str)
        ]
        review_content = ''.join(review_content_parts).replace('\n', '').strip()    
    else:
        review_content = "N/A"

    temp_rev_list.append(store_name)        # 가게 이름
    temp_rev_list.append(store_id)          # 카카오맵 ID
    temp_rev_list.append(reviewer_name)     # 리뷰어 이름
    temp_rev_list.append(reviewer_level)    # 리뷰어 레벨
    temp_rev_list.append(review_stats.get('후기', 'N/A'))       # 후기 개수
    temp_rev_list.append(review_stats.get('별점평균', 'N/A'))   # 별점 평균
    temp_rev_list.append(review_stats.get('팔로워', 'N/A'))     # 팔로워 수
    temp_rev_list.append(review_date)     # 리뷰 작성일
    temp_rev_list.append(rating)     # 별점
    temp_rev_list.append(review_content)    # 리뷰 내용

    # 7. 리뷰 사진 URL
    photo_urls = []
    # .list_photo 클래스를 가진 ul 태그 안의 모든 img 태그를 찾습니다.
    # CSS Selector: .list_photo img
    img_tags = temp_rev.select('.list_photo img.img_g')

    if img_tags:
        for img_tag in img_tags:
            # img 태그의 'src' 속성 값을 가져옵니다.
            if 'src' in img_tag.attrs:
                photo_url = img_tag['src']
                photo_urls.append(photo_url)

        temp_rev_list.append(photo_urls)                
    else:
        temp_rev_list.append("N/A")
            
    return temp_rev_list

### 코드 본문 시작점 ###

for url in url_list:        
    # ## 스크래핑용 브라우저 오픈
    # driver = webdriver.Chrome()

    driver.get(url)
    time.sleep(2)

    ## 초기 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # time.sleep(0.2)

    ## '후기'탭 유무 확인 후 있으면 클릭, 없으면 다음 가게로 패스
    if soup.find('a', href='#comment') or soup.find('a', string='후기'):
        # print("후기가 있습니다. 후기 탭을 엽니다.")
        driver.find_element(By.XPATH, '//*[@id="mainContent"]/div[2]/div[1]/div/ul/li[4]/a').send_keys(Keys.ENTER)
        time.sleep(0.2)
        # 후기 탭 열고 재파싱
        html = driver.page_source   
        soup = BeautifulSoup(html, 'html.parser')
        time.sleep(0.2)
    else:   # 후기 없을 시 패스
        # print("후기가 없습니다!")
        # driver.quit()
        # sys.exit()  # 실행 강제 종료
        continue

    ## 가게 이름 가져오기
    store_name_tag = soup.select('#mainContent > div.top_basic > div.info_main > div.unit_info > h3')
    store_name = store_name_tag[0].contents[1]
    store_id = url.split('/')[-1]

    ## 현재 스크래핑 중인 가게 이름 표시 ##
    print("### 사업장 이름 : ",store_name , "/ 카카오맵 상세정보 ID :",store_id , "###")

    ## 리뷰 그룹 가져오기
    review_group = soup.select('#mainContent > div.main_detail > div.detail_cont > div.section_comm.section_review > div.group_review > ul')
    # review_group[0].contents   # 리뷰들 모음. 한페이지에 기본 최대 20개
    # time.sleep(0.5)

    # ## 리뷰 저장용 DF 생성
    # review_df = pd.DataFrame(columns=['사업장명', '사업장ID', '리뷰어이름', '리뷰어레벨', '후기개수', '별점평균', '팔로워수', '작성일', '별점', '리뷰내용', '사진URL'])

    rev_cnt = 0     # 리뷰 수 카운터
    while review_group:
        if rev_cnt >= len(review_group[0].contents)-1:
            break

        temp_rev = review_group[0].contents[rev_cnt]
        # time.sleep(0.5)
        temp_rev_list = all_review_scraping(temp_rev, rev_cnt)   

        print(temp_rev_list)    # 스크래핑한 리뷰 리스트 출력
        review_df.loc[len(review_df)] = temp_rev_list   # 사전 지정 DF에 추가
        rev_cnt += 1
        # time.sleep(0.5)

    time.sleep(0.5)

print('** 리뷰 스크래핑 완료**')
driver.quit()

### 사업장 이름 :  EP 커피N바 / 카카오맵 상세정보 ID : 633908761 ###
['EP 커피N바', '633908761', '피트조아', '8', '7', '4.4', '0', '2023.07.19.', '5.0', '최고입니다', ['//img1.kakaocdn.net/cthumb/local/C280x280.q50/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flocal%2FkakaomapPhoto%2Freview%2F6bd3293cb6e38c0d8fd44c375b8207dfb2632fc2%3Foriginal']]
['EP 커피N바', '633908761', 'ppp', '17', '33', '3.6', '0', '2023.07.02.', '5.0', '맛은 물론이고 너무나 유쾌한 접객. 단골해야 해요', ['//img1.kakaocdn.net/cthumb/local/C280x280.q50/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flocal%2FkakaomapPhoto%2Freview%2Fe3534eb150c51822c216dea4cb8d143ece155c1a%3Foriginal']]
### 사업장 이름 :  순대일번지 / 카카오맵 상세정보 ID : 821235252 ###
### 사업장 이름 :  하이아시아 / 카카오맵 상세정보 ID : 2034930781 ###
['하이아시아', '2034930781', 'jess', '4', '7', '3.9', '0', '2025.05.08.', '5.0', '진짜 맛있어요,,,', ['//img1.kakaocdn.net/cthumb/local/C280x280.q50/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flocal%2FkakaomapPhoto%2Freview%2F4cf110e0905ed2cadbdd6562a6aa5471f05a01d4%3Foriginal', '//img1.kakaocdn.net/cthumb/local/C280x28

In [12]:
review_df

,사업장명,사업장ID,리뷰어이름,리뷰어레벨,후기개수,별점평균,팔로워수,작성일,별점,리뷰내용,사진URL
0,EP 커피N바,633908761,피트조아,8,7,4.4,0,2023.07.19.,5.0,최고입니다,[//img1.kakaocdn.net/cthumb/local/C280x280.q50...
1,EP 커피N바,633908761,ppp,17,33,3.6,0,2023.07.02.,5.0,맛은 물론이고 너무나 유쾌한 접객. 단골해야 해요,[//img1.kakaocdn.net/cthumb/local/C280x280.q50...
2,하이아시아,2034930781,jess,4,7,3.9,0,2025.05.08.,5.0,"진짜 맛있어요,,,",[//img1.kakaocdn.net/cthumb/local/C280x280.q50...
3,주실,1872261259,허제,5,11,5,1,2025.05.05.,5.0,뭐하나 빠지는게 없는 최고 맛집 술집,N/A
4,주실,1872261259,우주밖,2,2,5,0,2025.05.05.,5.0,생긴지 얼마 안됐는데 하이볼 이벤트도하고홀보는 직원분이 굉장히 친절하세요!!연남 오...,N/A
...,...,...,...,...,...,...,...,...,...,...,...
4464,코리아식당,15742085,혀니,34,141,2.8,8,2021.07.21.,5.0,사장님도 친절하시고 시원시원하시고연남동에서 잘 찾아볼 수 없는 orthodox 한식...,N/A
4465,코리아식당,15742085,Rena,23,66,3.4,4,2021.03.21.,1.0,홀에서 서빙하는 남자분 예의가 없음손님한테 명령조의 반말함,N/A
4466,코리아식당,15742085,5_4_1_2,12,19,3.9,0,2021.03.12.,5.0,밑반찬도 자주 바뀌고 맛있고 저녁에 술판나도 당당히 밥 먹을수 있는곳. 그리고 묘하...,N/A
4467,코리아식당,15742085,눈의띠네,38,669,3.9,17,2020.10.29.,3.0,찾아갈 맛집은 아니지만 무난한 밥집늦은 시간 연남에서 밥이 고플 때 거의 유일한 선택지,N/A


In [ ]:
## csv 파일로 저장
review_df.to_csv("kakaomap_yeonnam_review_scrap.csv", index=False, encoding="utf-8-sig")
